In [ ]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
import numpy as np

# Load the dataset
thailand_data = pd.read_csv('thailand.csv')

In [ ]:
# Data preprocessing
thailand_occupancy = thailand_data[thailand_data['variable'] == 'occupancy_rate']
choburi_occupancy = thailand_occupancy[thailand_occupancy['province_eng'] == 'Chonburi']
choburi_occupancy['date'] = pd.to_datetime(choburi_occupancy['date'])

# Time series analysis for occupancy rate
choburi_occupancy.set_index('date', inplace=True)
choburi_occupancy = choburi_occupancy.resample('M').mean()

# Visualizing the data
plt.figure(figsize=(12, 6))
plt.plot(choburi_occupancy.index, choburi_occupancy['value'], label='Occupancy Rate', color='blue')
plt.title('Occupancy Rate in Chonburi')
plt.xlabel('Date')
plt.ylabel('Occupancy Rate')
plt.legend()
plt.show()

# ARIMA Model
# Differencing the data
choburi_occupancy_diff = choburi_occupancy['value'].diff().dropna()

# ACF and PACF plots
plot_acf(choburi_occupancy_diff)
plot_pacf(choburi_occupancy_diff)
plt.show()

# Fit ARIMA model
model = SARIMAX(choburi_occupancy['value'], order=(1, 1, 1), seasonal_order=(1, 1, 0, 12))
results = model.fit()

# Forecasting
forecast = results.get_forecast(steps=12)
forecast_index = pd.date_range(start=choburi_occupancy.index[-1], periods=13, freq='M')[1:]
forecast_values = forecast.predicted_mean

# Plotting the forecast
plt.figure(figsize=(12, 6))
plt.plot(choburi_occupancy.index, choburi_occupancy['value'], label='Historical Occupancy Rate', color='blue')
plt.plot(forecast_index, forecast_values, label='Forecasted Occupancy Rate', color='orange')
plt.fill_between(forecast_index, 
                 forecast.conf_int()['lower value'], 
                 forecast.conf_int()['upper value'], color='pink', alpha=0.3)
plt.title('Occupancy Rate Forecast for Chonburi')
plt.xlabel('Date')
plt.ylabel('Occupancy Rate')
plt.legend()
plt.show()

# Comparing with the tourist data
thailand_tourist = thailand_data[thailand_data['variable'] == 'no_tourist_all']
choburi_tourist = thailand_tourist[thailand_tourist['province_eng'] == 'Chonburi']
choburi_tourist['date'] = pd.to_datetime(choburi_tourist['date'])
choburi_tourist.set_index('date', inplace=True)
choburi_tourist = choburi_tourist.resample('M').mean()

plt.figure(figsize=(12, 6))
plt.plot(choburi_tourist.index, choburi_tourist['value'], label='Number of Tourists', color='green')
plt.title('Number of Tourists in Chonburi')
plt.xlabel('Date')
plt.ylabel('Number of Tourists')
plt.legend()
plt.show()

# Repeat similar analysis for revenue
thailand_net_profit = thailand_data[thailand_data['variable'] == 'net_profit_all']
choburi_net_profit = thailand_net_profit[thailand_net_profit['province_eng'] == 'Chonburi']
choburi_net_profit['date'] = pd.to_datetime(choburi_net_profit['date'])
choburi_net_profit.set_index('date', inplace=True)

plt.figure(figsize=(12, 6))
plt.plot(choburi_net_profit.index, choburi_net_profit['value'], label='Net Profit', color='purple')
plt.title('Net Profit in Chonburi')
plt.xlabel('Date')
plt.ylabel('Net Profit')
plt.legend()
plt.show()